In [166]:
import pandas as pd
import numpy as np
import string
import re

In [183]:
data_df = pd.read_csv('data.csv', encoding = "ISO-8859-1")
kamus_slang = pd.read_csv('new_kamusalay.csv', header=None, names=['slang', 'baku'], encoding="ISO-8859-1")
abusive_words = pd.read_csv('abusive.csv', encoding="ISO-8859-1")['ABUSIVE'].tolist()

In [184]:
# Menghilangkan kata "USER" dan "RT"
data_df['Tweet'] = data_df['Tweet'].replace(r'USER', '', regex=True)
data_df['Tweet'] = data_df['Tweet'].replace(r'RT', '', regex=True)

#Membuat semua huruf menjadi lowercase
data_df['Tweet'] = data_df['Tweet'].str.lower()

#Menghapus semua tanda baca
data_df['Tweet'] = data_df['Tweet'].str.replace(r'[^\w\s]', '', regex=True)

#Menghapus angka
data_df['Tweet'] = data_df['Tweet'].str.replace(r'\d', '', regex=True)

#Menhapus spasi ekstra
data_df['Tweet'] = data_df['Tweet'].str.replace(r'\s+', ' ', regex=True)

#Menghapus tanda baca berulang
data_df['Tweet'] = data_df['Tweet'].str.replace(r'(?:(\.)\1+)+', r'\1', regex=True)

#Menghapus emoji
data_df['Tweet'] = data_df['Tweet'].str.replace(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', '', regex=True)

#Menghapus emoticon
data_df['Tweet'] = data_df['Tweet'].str.replace(r'[\U00002600-\U000027BF\U0001f300-\U0001f64F\U0001f680-\U0001f6FF\U0001f700-\U0001f77F\U0001f780-\U0001f7FF\U0001f800-\U0001f8FF\U0001f900-\U0001f9FF\U0001fa00-\U0001fa6F\U0001fa70-\U0001faFF\U00002190-\U000021FF]+', '', regex=True)

#Menghapus duplikasi
data_df.drop_duplicates(subset=['Tweet'], keep='first', inplace=True)

In [187]:
#Mengganti kata slang ke kata baku
def ganti_kata_baku(tweet):
    words = tweet.split()
    updated_words = []
    
    for word in words:
        replacement = kamus_slang[kamus_slang['slang'] == word]['baku'].values
        updated_words.append(replacement[0] if replacement else word)

    return ' '.join(updated_words)

# Terapkan fungsi pada kolom Tweet
data_df['Tweet'] = data_df['Tweet'].apply(ganti_kata_baku)

/var/folders/4_/qbxc6_011s35mgxlm4k342lh0000gn/T/ipykernel_79098/1827897581.py:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  updated_words.append(replacement[0] if replacement else word)


In [188]:
data_df['Tweet'] = data_df['Tweet'].apply(lambda x: ' '.join(['' if word in abusive_words else word for word in x.split()]))

In [191]:
df = data_df[['Tweet']]

In [192]:
df.to_csv('data_cleaned_final.csv', index=False)